In [33]:
import numpy as np

In [41]:
filename = 'data/validation.txt'
with open(filename, 'r') as f:
    data = f.read()

In [42]:
chars = list(set(data))
V = vocab_size = len(chars)

In [43]:
data_size = len(data)
len(data)

19349777

In [45]:
print('data has %d characters, %d unique.' % (data_size, vocab_size))

data has 19349777 characters, 101 unique.


In [44]:
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i, ch in enumerate(chars)}
print('char_to_ix', char_to_ix)
print('ix_to_char', ix_to_char)

char_to_ix {'Â': 0, 'm': 1, '2': 2, 'I': 3, 'C': 4, "'": 5, 'q': 6, '\n': 7, 'R': 8, 'P': 9, 'c': 10, 'E': 11, '?': 12, 'M': 13, 'V': 14, 'O': 15, ' ': 16, 'k': 17, '4': 18, '3': 19, 'l': 20, 'i': 21, '1': 22, 'T': 23, 'â': 24, 'N': 25, 'r': 26, 'Ã': 27, 'y': 28, '0': 29, 'D': 30, '´': 31, '7': 32, '€': 33, '“': 34, 'a': 35, 'e': 36, '9': 37, '*': 38, 'U': 39, 'p': 40, 'z': 41, ';': 42, 'Q': 43, 'u': 44, '<': 45, 'Š': 46, 'X': 47, 'n': 48, 'J': 49, 'h': 50, 'w': 51, 'L': 52, '˜': 53, 'Ž': 54, 't': 55, '"': 56, ')': 57, '.': 58, 'b': 59, ',': 60, '-': 61, '‹': 62, '$': 63, '/': 64, 'f': 65, 'j': 66, 'Y': 67, 'x': 68, '!': 69, '”': 70, 's': 71, 'B': 72, 'K': 73, 'v': 74, 'A': 75, 'ð': 76, ':': 77, '5': 78, 'G': 79, '&': 80, '+': 81, '©': 82, '™': 83, 'd': 84, 'S': 85, '6': 86, '#': 87, 'H': 88, 'œ': 89, 'Ÿ': 90, 'W': 91, '8': 92, '(': 93, 'g': 94, '±': 95, '¦': 96, '\xad': 97, 'o': 98, 'Z': 99, 'F': 100}
ix_to_char {0: 'Â', 1: 'm', 2: '2', 3: 'I', 4: 'C', 5: "'", 6: 'q', 7: '\n', 8: 'R',

In [46]:
# Size of hidden state vectors; applies to h and c.
H = hidden_size = 100
seq_length = 16 # number of steps to unroll the LSTM for
learning_rate = 0.1

In [47]:
# The input x is concatenated with state h
# The joined vector is used to feed into most blocks within the LSTM cell
HV = H + V # (100 + 65)

In [48]:
HV

201

In [49]:
MAX_DATA = 1000000

In [50]:
# Model parameters/weights -- these are shared among all steps
# Inputs are characters one-hot encoded in a vocab-sized vector. (1, vocab_size)
# Dimensions: H = hidden_size, V = vocab_size, HV = hidden_size + vocab_size

Wf = np.random.randn(H, HV) * 0.01 # (100, 165)
bf = np.zeros((H, 1)) # (100, 1)
Wi = np.random.randn(H, HV) * 0.01 # (100, 165)
bi = np.zeros((H, 1)) # (100, 1)
Wcc = np.random.randn(H, HV) * 0.01 # (100, 165)
bcc = np.zeros((H, 1)) # (100, 1)
Wo = np.random.randn(H, HV) * 0.01 # (100, 165)
bo = np.zeros((H, 1)) # (100, 1)
Wy = np.random.randn(V, H) * 0.01 # (65, 100)
by = np.zeros((V, 1)) # (100, 1)

In [51]:
def sigmoid(z):
    """Computes sigmoid function.

    z: array of input values.

    Returns array of outputs, sigmoid(z).
    """
    # Note: this version of sigmoid tries to avoid overflows in the computation
    # of e^(-z), by using an alternative formulation when z is negative, to get
    # 0. e^z / (1+e^z) is equivalent to the definition of sigmoid, but we won't
    # get e^(-z) to overflow when z is very negative.
    # Since both the x and y arguments to np.where are evaluated by Python, we
    # may still get overflow warnings for large z elements; therefore we ignore
    # warnings during this computation.
    with np.errstate(over='ignore', invalid='ignore'):
        return np.where(z >= 0,
                        1 / (1 + np.exp(-z)),
                        np.exp(z) / (1 + np.exp(z)))

In [52]:
def lossFun(inputs, targets, hprev, cprev):
    """Runs forward and backward passes through the RNN.
      inputs, targets: Lists of integers. For some i, inputs[i] is the input
                       character (encoded as an index into the ix_to_char map)
                       and targets[i] is the corresponding next character in the
                       training data (similarly encoded).
      hprev: Hx1 array of initial hidden state (column vector)
      cprev: Hx1 array of initial hidden state (column vector)
      returns: loss, gradients on model parameters, and last hidden states
    """
    # Caches that keep values computed in the forward pass at each time step, to be reused in the backward pass.
    # Initialize as dictionaries to be indexed with time steps
    xs, xhs, ys, ps, hs, cs, fgs, igs, ccs, ogs = (
            {}, {}, {}, {}, {}, {}, {}, {}, {}, {})

    # Initial incoming states (from time step = -1)
    hs[-1] = np.copy(hprev)
    cs[-1] = np.copy(cprev)

    loss = 0
    
    # Forward pass
    for t in range(len(inputs)):
        # Input at time step t is xs[t]. Prepare a one-hot encoded vector of
        # shape (V, 1). inputs[t] is the index where the 1 goes.
        xs[t] = np.zeros((V, 1)) # (65, 1) 
        xs[t][inputs[t]] = 1 # (65, 1) -> column vector

        # hprev and xs[t] are column vector; stack them together into a "taller"
        # column vector - first the elements of x, then h.
        xhs[t] = np.vstack((xs[t], hs[t-1])) # (165, 1) -> column vector

        # Gates f, i and o
        fgs[t] = sigmoid(np.dot(Wf, xhs[t]) + bf) # (100, 165) @ (165, 1) = (100, 1)
        igs[t] = sigmoid(np.dot(Wi, xhs[t]) + bi) # (100, 165) @ (165, 1) = (100, 1)
        ogs[t] = sigmoid(np.dot(Wo, xhs[t]) + bo) # (100, 165) @ (165, 1) = (100, 1)

        # Candidate cc
        ccs[t] = np.tanh(np.dot(Wcc, xhs[t]) + bcc) # (100, 165) @ (165, 1) = (100, 1)

        # This step's h and c
        cs[t] = fgs[t] * cs[t-1] + igs[t] * ccs[t] # (100, 1) * (100, 1) + (100, 1) * (100, 1)
        hs[t] = np.tanh(cs[t]) * ogs[t] # (100, 1)

        # Convert to output
        ys[t] = np.dot(Wy, hs[t]) + by # (65, 100) @ (100, 1) = (65, 1)
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # (65, 1)

        # Cross-entropy loss.
        loss += -np.log(ps[t][targets[t], 0]) # (65, 1)

    # Initialize gradients of all weights/biases to 0.
    # We only need to compute 5 matrices in backpropogation!
    dWf = np.zeros_like(Wf)
    dbf = np.zeros_like(bf)
    
    dWi = np.zeros_like(Wi)
    dbi = np.zeros_like(bi)
    
    dWcc = np.zeros_like(Wcc)
    dbcc = np.zeros_like(bcc)
    
    dWo = np.zeros_like(Wo)
    dbo = np.zeros_like(bo)
    
    dWy = np.zeros_like(Wy)
    dby = np.zeros_like(by)

    # Incoming gradients for h and c; for backwards loop step these represent
    # dh[t] and dc[t]; we do truncated BPTT, so assume they are 0 initially.
    dhnext = np.zeros_like(hs[0])
    dcnext = np.zeros_like(cs[0])
    
    # The backwards pass iterates over the input sequence backwards.
    for t in reversed(range(len(inputs))):
        # Backprop through the gradients of loss and softmax.
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1

        # Compute gradients for the Wy and by parameters.
        dWy += np.dot(dy, hs[t].T)
        dby += dy

        # Backprop through the fully-connected layer (Wy, by) to h. Also add up
        # the incoming gradient for h from the next cell.
        dh = np.dot(Wy.T, dy) + dhnext

        # Backprop through multiplication with output gate; here "dtanh" means
        # the gradient at the output of tanh.
        dctanh = ogs[t] * dh
        # Backprop through the tanh function; since cs[t] branches in two
        # directions we add dcnext too.
        dc = dctanh * (1 - np.tanh(cs[t]) ** 2) + dcnext

        # Backprop through multiplication with the tanh; here "dhogs" means
        # the gradient at the output of the sigmoid of the output gate. Then
        # backprop through the sigmoid itself (ogs[t] is the sigmoid output).
        dhogs = dh * np.tanh(cs[t])
        dho = dhogs * ogs[t] * (1 - ogs[t])

        # Compute gradients for the output gate parameters.
        dWo += np.dot(dho, xhs[t].T)
        dbo += dho

        # Backprop dho to the xh input.
        dxh_from_o = np.dot(Wo.T, dho)

        # Backprop through the forget gate: sigmoid and elementwise mul.
        dhf = cs[t-1] * dc * fgs[t] * (1 - fgs[t])
        dWf += np.dot(dhf, xhs[t].T)
        dbf += dhf
        dxh_from_f = np.dot(Wf.T, dhf)

        # Backprop through the input gate: sigmoid and elementwise mul.
        dhi = ccs[t] * dc * igs[t] * (1 - igs[t])
        dWi += np.dot(dhi, xhs[t].T)
        dbi += dhi
        dxh_from_i = np.dot(Wi.T, dhi)

        dhcc = igs[t] * dc * (1 - ccs[t] ** 2)
        dWcc += np.dot(dhcc, xhs[t].T)
        dbcc += dhcc
        dxh_from_cc = np.dot(Wcc.T, dhcc)

        # Combine all contributions to dxh, and extract the gradient for the
        # h part to propagate backwards as dhnext.
        dxh = dxh_from_o + dxh_from_f + dxh_from_i + dxh_from_cc
        dhnext = dxh[V:, :]

        # dcnext from dc and the forget gate.
        dcnext = fgs[t] * dc

    # Gradient clipping to the range [-5, 5].
    for dparam in [dWf, dbf, dWi, dbi, dWcc, dbcc, dWo, dbo, dWy, dby]:
        np.clip(dparam, -5, 5, out=dparam)

    return (loss, dWf, dbf, dWi, dbi, dWcc, dbcc, dWo, dbo, dWy, dby,
            hs[len(inputs)-1], cs[len(inputs)-1])

In [54]:
def sample(h, c, seed_ix, n):
    """Sample a sequence of integers from the model.

    Runs the LSTM in forward mode for n steps; seed_ix is the seed letter for
    the first time step, h and c are the memory state. Returns a sequence of
    letters produced by the model (indices).
    """
    x = np.zeros((V, 1))
    x[seed_ix] = 1
    ixes = []

    for t in range(n):
        # Run the forward pass only.
        xh = np.vstack((x, h))
        fg = sigmoid(np.dot(Wf, xh) + bf)
        ig = sigmoid(np.dot(Wi, xh) + bi)
        og = sigmoid(np.dot(Wo, xh) + bo)
        cc = np.tanh(np.dot(Wcc, xh) + bcc)
        c = fg * c + ig * cc
        h = np.tanh(c) * og
        y = np.dot(Wy, h) + by
        p = np.exp(y) / np.sum(np.exp(y))

        # Sample from the distribution produced by softmax.
        ix = np.random.choice(range(V), p=p.ravel())
        x = np.zeros((V, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [55]:
def gradCheck(inputs, targets, hprev, cprev):
    global Wf, Wi, bf, bi, Wcc, bcc, Wo, bo, Wy, by
    num_checks, delta = 10, 1e-5
    (_, dWf, dbf, dWi, dbi, dWcc, dbcc, dWo, dbo, dWy, dby,
     _, _) = lossFun(inputs, targets, hprev, cprev)
    for param, dparam, name in zip(
            [Wf, bf, Wi, bi, Wcc, bcc, Wo, bo, Wy, by],
            [dWf, dbf, dWi, dbi, dWcc, dbcc, dWo, dbo, dWy, dby],
            ['Wf', 'bf', 'Wi', 'bi', 'Wcc', 'bcc', 'Wo', 'bo', 'Wy', 'by']):
        assert dparam.shape == param.shape
        print(name)
        for i in range(num_checks):
            ri = np.random.randint(0, param.size)
            old_val = param.flat[ri]
            param.flat[ri] = old_val + delta
            numloss0 = lossFun(inputs, targets, hprev, cprev)[0]
            param.flat[ri] = old_val - delta
            numloss1 = lossFun(inputs, targets, hprev, cprev)[0]
            param.flat[ri] = old_val # reset
            grad_analytic = dparam.flat[ri]
            grad_numerical = (numloss0 - numloss1) / (2 * delta)
            if grad_numerical + grad_analytic == 0:
                rel_error = 0
            else:
                rel_error = (abs(grad_analytic - grad_numerical) /
                             abs(grad_numerical + grad_analytic))
            print('%s, %s => %e' % (grad_numerical, grad_analytic, rel_error))


In [56]:
def basicGradCheck():
    inputs = [char_to_ix[ch] for ch in data[:seq_length]]
    targets = [char_to_ix[ch] for ch in data[1:seq_length+1]]
    hprev = np.random.randn(H, 1)
    cprev = np.random.randn(H, 1)
    gradCheck(inputs, targets, hprev, cprev)

# Uncomment this to run gradient checking instead of training
#basicGradCheck()
#sys.exit()

# n is the iteration counter; p is the input sequence pointer, at the beginning
# of each step it points at the sequence in the input that will be used for
# training this iteration.
n, p = 0, 0

# Memory variables for Adagrad.
mWf = np.zeros_like(Wf)
mbf = np.zeros_like(bf)
mWi = np.zeros_like(Wi)
mbi = np.zeros_like(bi)
mWcc = np.zeros_like(Wcc)
mbcc = np.zeros_like(bcc)
mWo = np.zeros_like(Wo)
mbo = np.zeros_like(bo)
mWy = np.zeros_like(Wy)
mby = np.zeros_like(by)
smooth_loss = -np.log(1.0/V) * seq_length

while p < MAX_DATA:
    # Prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0:
        # Reset RNN memory
        hprev = np.zeros((H, 1))
        cprev = np.zeros((H, 1))
        p = 0 # go from start of data

    # In each step we unroll the RNN for seq_length cells, and present it with
    # seq_length inputs and seq_length target outputs to learn.
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # Sample from the model now and then.
    if n % 1000 == 0:
        sample_ix = sample(hprev, cprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))

    # Forward seq_length characters through the RNN and fetch gradient.
    (loss, dWf, dbf, dWi, dbi, dWcc, dbcc, dWo, dbo, dWy, dby,
     hprev, cprev) = lossFun(inputs, targets, hprev, cprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 200 == 0:
        print('iter %d (p=%d), loss %f' % (n, p, smooth_loss))

    # Perform parameter update with Adagrad.
    for param, dparam, mem in zip(
            [Wf, bf, Wi, bi, Wcc, bcc, Wo, bo, Wy, by],
            [dWf, dbf, dWi, dbi, dWcc, dbcc, dWo, dbo, dWy, dby],
            [mWf, mbf, mWi, mbi, mWcc, mbcc, mWo, mbo, mWy, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)

    p += seq_length
    n += 1


----
 kŠŸ#f0jX)KOX¦PD˜<f,Oœ/<pyl&Gc2CâŽ€QX(K&rk.Š,'su!K6˜O‹8mpexo"t4Q,V(Y±©v”-oRCTfH.˜¦1*vŽXIE!#±aHf.âBV+g6˜j™+e¦swxc?©.8uSÃŠ#:ld4Ksðcg/MEw
n8”!Uv€ð”œM2#Jy™.yjZJÃ&0my1I38?P2m,m-ÂSemPŠp:A”&+ycNœt ÃjgB;"
jâT“ 
----
iter 0 (p=0), loss 73.841930
iter 200 (p=3200), loss 68.078371
iter 400 (p=6400), loss 61.791523
iter 600 (p=9600), loss 56.126380
iter 800 (p=12800), loss 51.381767
----
 . Tht aloogd Fos. She purlid to it to gaike mot foll ball soot all relpetola, alt. L to very thiy loveds, to he ca 'not bit alled tim ttie tot namg lammd the ander ailt, sange of and all. Tam ans vely 
----
iter 1000 (p=16000), loss 47.197665
iter 1200 (p=19200), loss 43.757579
iter 1400 (p=22400), loss 40.694380
iter 1600 (p=25600), loss 38.098688
iter 1800 (p=28800), loss 36.109759
----
  But a now bould in him kimn the wut him smikedn remhen wister has pVme. Timty hemmed theret hw the and the cakrow tild fart put moy bech. Timcy thos was dald bot hamy and him trainke momed dik mV. He 
----
iter 2000 (p=32